https://towardsdatascience.com/predicting-the-music-mood-of-a-song-with-deep-learning-c3ac2b45229e

https://realpython.com/python-nltk-sentiment-analysis/

MEDIUM : https://towardsdatascience.com/how-to-analyze-emotions-and-words-of-the-lyrics-from-your-favorite-music-artist-bbca10411283

In [1]:
#libraries used to extract, clean and manipulate the datafrom helpers import *

import pandas as pd
import numpy as np
import string#To plot the graphsfrom wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn')#library used to count the frequency of wordsfrom sklearn.feature_extraction.text import CountVectorizer#To create the sentiment analysis model, tokenization and lemmatizationimport nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
import nltk.data
nltk.download('vader_lexicon')
nltk.download('punkt')

import time

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#Spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

spotify_client_id = "3a2f65eb20164b24b59b7fdd6876bb51"
spotify_client_secret = "9f6d3a070cdf4eb8adda14a794d60469"

#Genius credentials
genius_token = "4SoLPv5UHyFZyAe5T-PhxS-bRCRrdNIgMNpeqjjIaWvnRbUmTpoGxd9knxBtlQyg"

#Authentication spotify - without user
client_credentials_manager = SpotifyClientCredentials(client_id="3a2f65eb20164b24b59b7fdd6876bb51", client_secret="9f6d3a070cdf4eb8adda14a794d60469")
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
#playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbL3J0k32lWnN?si=5c3f6128e25049d9"  #top50DK
playlist_link ="https://open.spotify.com/playlist/4XDFklOPJOJp3geewWo6vo?si=97b4324becef4027"  #Small 6 song playlist
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [4]:
for track in sp.playlist_tracks(playlist_URI)["items"]:
    
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    
    #Year
    track_year = track["track"]["album"]["release_date"]
    
    print(track_year)
    #Printin details
    #print(track_name,"\n", 
    #      artist_name,"\n",
    #      #sp.audio_features(track_uri)[0], "\n", 
    #      track_uri
    #     )

2019-06-28
2020-04-03
2021-05-14
2019-07-19
2021-08-16


In [5]:
sp.audio_features("https://open.spotify.com/track/7MXVkk9YMctZqd1Srtv4MB?si=e11b8d5376554fde")[0]

{'danceability': 0.679,
 'energy': 0.587,
 'key': 7,
 'loudness': -7.015,
 'mode': 1,
 'speechiness': 0.276,
 'acousticness': 0.141,
 'instrumentalness': 6.35e-06,
 'liveness': 0.137,
 'valence': 0.486,
 'tempo': 186.003,
 'type': 'audio_features',
 'id': '7MXVkk9YMctZqd1Srtv4MB',
 'uri': 'spotify:track:7MXVkk9YMctZqd1Srtv4MB',
 'track_href': 'https://api.spotify.com/v1/tracks/7MXVkk9YMctZqd1Srtv4MB',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7MXVkk9YMctZqd1Srtv4MB',
 'duration_ms': 230453,
 'time_signature': 4}

In [17]:
data = pd.read_csv("../data/cleanedDatasetWithFeatures.csv")

In [18]:
data.head()

,Unnamed: 0,Unnamed: 0.1,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0776,0.187,0.000030,0.1590,0.9070,102.034,195840.0
1,1,1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,0.1700,0.209,0.000000,0.1120,0.8460,177.833,228200.0
2,2,2,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0470,0.426,0.000001,0.1440,0.4640,81.922,274000.0
3,3,3,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.2140,0.542,0.000013,0.0595,0.4420,96.507,417920.0
4,4,4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,0.0320,0.423,0.000002,0.4350,0.0979,129.863,221538.0


## Adding features of each song to the dataset

In [10]:
test_set = data.head()

In [11]:
test_set.shape

(5, 20)

In [ ]:
# Add columns to existing dataset with the audio features.
# Implementing a timer to check the time for adding new columns.
import time
tic = time.perf_counter()

errors = 0

for index, row in data.iterrows():
    try:
        current = sp.audio_features(row['url'])[0]
        data.at[index,'danceability'] = float(current['danceability'])
        data.at[index,'energy'] = current['energy']
        data.at[index,'key'] = current['key']
        data.at[index,'loudness'] = current['loudness']
        data.at[index,'mode'] = current['mode']
        data.at[index,'speechiness'] = current['speechiness']
        data.at[index,'acousticness'] = current['acousticness']
        data.at[index,'instrumentalness'] = current['instrumentalness']
        data.at[index,'liveness'] = current['liveness']
        data.at[index,'valence'] = current['valence']
        data.at[index,'tempo'] = current['tempo']
        data.at[index,'duration_ms'] = current['duration_ms']
        print(str(int(((index+1)/data.shape[0])*100)) + '% done - ' + str(index+1) + ' of ' + str(data.shape[0]) + ' songs iterated - ' + errors + ' encountered.', end='\r')
    except:
        #append error or insert specific values?
        errors = errors+1
        pass
    
        
toc = time.perf_counter()
print(f"Added new rows from external links in {toc - tic:0.4f} seconds")

In [16]:
data.isnull().sum()

Unnamed: 0          0
Unnamed: 0.1        0
title               0
rank                0
date                0
artist              0
url                 0
region              0
streams             0
danceability        0
energy              0
key                 0
loudness            0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
dtype: int64

**Missing values from iteration:**

In [ ]:
'''
6119
20101
23710 
25962
33986
34521 
38726 
39988 
52686
63662 
64375
66721
72738
77099 
94502 
96669 
99117 
'''

In [ ]:
data[6118:6120]

In [11]:
#Saving the sub dataset for testing.
from pathlib import Path
path = Path('../data/cleanedDatasetWithFeatures.csv')
path.parent.mkdir(parents=True, exist_ok=True)
data.to_csv(path)

In [ ]:
new_data = data.head()

In [ ]:
new_data.head()

## Fetching genres from genius

In [2]:
import csv
import requests
import json
import re
import time
from pandas.io.json import json_normalize
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [20]:
data = pd.read_csv("../data/cleanedDatasetWithFeatures.csv")

In [23]:
data.head()

,Unnamed: 0,Unnamed: 0.1,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0776,0.187,0.000030,0.1590,0.9070,102.034,195840.0
1,1,1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,0.1700,0.209,0.000000,0.1120,0.8460,177.833,228200.0
2,2,2,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0470,0.426,0.000001,0.1440,0.4640,81.922,274000.0
3,3,3,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.2140,0.542,0.000013,0.0595,0.4420,96.507,417920.0
4,4,4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,0.0320,0.423,0.000002,0.4350,0.0979,129.863,221538.0


In [6]:
data = data.dropna()

In [6]:
def get_track_name(nr):
    track_string = str(data.title[nr])
    x = track_string.find(" (")
    y = track_string.find(" -")
    
    if(int(x) < 0):
        track_string = track_string
    if(int(x) > 0):
        track_string = track_string[:x]
    if (int(y) > 0):
        track_string = track_string[:y]
    
    return track_string

def get_track_artist(nr):
    
    artist_string = str(data.artist[nr])

    y = artist_string.find(", ")
    x = artist_string.find(" (")
    
    if(int(x) > 0):
        artist_string = artist_string.replace("' ", "")

    if(int(y) > 0):
        #artist_string = y
        artist_string = artist_string[:y]     
        
    return artist_string

In [7]:
def get_url(track_nr):
    track = re.sub('[^a-zA-Z0-9 \n\.]', '', str(get_track_name(track_nr)))
    artist = re.sub('[^a-zA-Z0-9 \n\.]', '', str(get_track_artist(track_nr)))
    
    if (int(artist.find(" ")) > 0):
        artist = artist.replace(" ", "-")
        
    if (int(track.find(" ")) > 0):
        track = track.replace(" ", "-")
    
    url_string = str('https://genius.com/' + artist + '-' + track + '-lyrics')
    
    return str(url_string)

In [8]:
def extract_credits(URL):
        
    r = requests.get(URL)
    
    if (int(r.status_code) == 200):
        
        genre = []

        # Parsing the HTML
        soup = BeautifulSoup(r.content, 'html.parser')
        g = soup.find(class_='SongTags__Container-xixwg3-1 bZsZHM')
        ggenre = g.find('a')        
                    
    else:
        return ''
        
        
    return ggenre.text

In [19]:
data.drop([0,20000], axis=0, inplace=True)

KeyError: '[0, 20000] not found in axis'

In [13]:
extract_credits(get_url(data.iloc[50][0]))

'Pop'

In [14]:
get_url(data.iloc[5][0])

'https://genius.com/Ed-Sheeran-Shape-of-You-lyrics'

In [25]:
data.drop('Unnamed: 0', inplace=True, axis=1)
data.drop('Unnamed: 0.1', inplace=True, axis=1)

In [26]:
data

,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0776,0.1870,0.000030,0.1590,0.9070,102.034,195840.0
1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,0.1700,0.2090,0.000000,0.1120,0.8460,177.833,228200.0
2,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0470,0.4260,0.000001,0.1440,0.4640,81.922,274000.0
3,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.2140,0.5420,0.000013,0.0595,0.4420,96.507,417920.0
4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,0.0320,0.4230,0.000002,0.4350,0.0979,129.863,221538.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104539,Coraz bliżej Święta,200,2018-11-23,Nosowska,https://open.spotify.com/track/2K6hdg6MKfKrDQB...,Poland,9718.0,0.639,0.727,5.0,-6.985,0.0587,0.0939,0.013200,0.0929,0.4470,146.081,205687.0
104540,Ik Heb Je,200,2018-10-26,Maan,https://open.spotify.com/track/2Xqs5ozueApOYZm...,Netherlands,25204.0,0.845,0.559,6.0,-5.171,0.1680,0.3370,0.001710,0.0758,0.6280,100.242,176404.0
104541,המנון הלוחם,200,2021-04-23,Hatikva 6,https://open.spotify.com/track/2i1xX8VmwMDtmF7...,Israel,3822.0,0.648,0.668,1.0,-10.186,0.0879,0.0722,0.000000,0.1300,0.8480,134.058,195376.0
104542,ครางชื่ออ้ายแน,200,2018-10-27,ศรีจันทร์ วีสี,https://open.spotify.com/track/5ZwxJQ8vR5cHWer...,Thailand,2390.0,0.488,0.687,0.0,-8.530,0.0503,0.1540,0.000001,0.1330,0.5530,153.052,215427.0


In [41]:
sub_data = data[:100]

In [21]:
# Add genres to the existing dataset.

tic = time.perf_counter()

errors = 0

for index, row in data.iterrows():
    print(str(int(((index+1)/data.shape[0])*100)) + '% done - ' + str(index+1) + ' of ' + str(data.shape[0]) + ' songs iterated - ' + str(errors) + ' null values encountered.', end='\r')
    data.at[index,'genre'] = extract_credits(get_url(index))

toc = time.perf_counter()
print(f"Added new rows from external links in {toc - tic:0.4f} seconds")

Added new rows from external links in 86981.9215 secondsues encountered.


In [22]:
# Check how many values are null.
data['genre'].isnull().sum()

0

In [23]:
df_with_genre = data[data['genre'].notna()]

In [24]:
df_with_genre

,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genre
1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,0.1700,0.2090,0.000000,0.1120,0.8460,177.833,228200.0,Pop
2,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0470,0.4260,0.000001,0.1440,0.4640,81.922,274000.0,
3,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.2140,0.5420,0.000013,0.0595,0.4420,96.507,417920.0,
4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,0.0320,0.4230,0.000002,0.4350,0.0979,129.863,221538.0,Pop
5,7 rings,1,2019-03-16,Ariana Grande,https://open.spotify.com/track/6ocbgoVGwYJhOv1...,Indonesia,125837.0,0.778,0.317,1.0,-10.732,0.3340,0.5920,0.000000,0.0881,0.3270,140.048,178627.0,Rap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104556,Coraz bliżej Święta,200,2018-11-23,Nosowska,https://open.spotify.com/track/2K6hdg6MKfKrDQB...,Poland,9718.0,0.639,0.727,5.0,-6.985,0.0587,0.0939,0.013200,0.0929,0.4470,146.081,205687.0,
104557,Ik Heb Je,200,2018-10-26,Maan,https://open.spotify.com/track/2Xqs5ozueApOYZm...,Netherlands,25204.0,0.845,0.559,6.0,-5.171,0.1680,0.3370,0.001710,0.0758,0.6280,100.242,176404.0,Pop
104558,המנון הלוחם,200,2021-04-23,Hatikva 6,https://open.spotify.com/track/2i1xX8VmwMDtmF7...,Israel,3822.0,0.648,0.668,1.0,-10.186,0.0879,0.0722,0.000000,0.1300,0.8480,134.058,195376.0,
104559,ครางชื่ออ้ายแน,200,2018-10-27,ศรีจันทร์ วีสี,https://open.spotify.com/track/5ZwxJQ8vR5cHWer...,Thailand,2390.0,0.488,0.687,0.0,-8.530,0.0503,0.1540,0.000001,0.1330,0.5530,153.052,215427.0,


In [25]:
df_with_genre = df_with_genre[df_with_genre['genre'] != '']

In [26]:
df_with_genre.reset_index()

,index,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genre
0,1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,0.1700,0.2090,0.000000,0.1120,0.8460,177.833,228200.0,Pop
1,4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,0.0320,0.4230,0.000002,0.4350,0.0979,129.863,221538.0,Pop
2,5,7 rings,1,2019-03-16,Ariana Grande,https://open.spotify.com/track/6ocbgoVGwYJhOv1...,Indonesia,125837.0,0.778,0.317,1.0,-10.732,0.3340,0.5920,0.000000,0.0881,0.3270,140.048,178627.0,Rap
3,6,Shape of You,1,2017-03-08,Ed Sheeran,https://open.spotify.com/track/7qiZfU4dY1lWllz...,Portugal,55132.0,0.825,0.652,1.0,-3.183,0.0802,0.5810,0.000000,0.0931,0.9310,95.977,233713.0,R&B
4,9,Someone You Loved,1,2019-03-16,Lewis Capaldi,https://open.spotify.com/track/2TIlqbIneP0ZY1O...,Ireland,63074.0,0.501,0.405,1.0,-5.679,0.0319,0.7510,0.000000,0.1050,0.4460,109.891,182161.0,Pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57367,104548,One More I Love You,200,2021-06-14,Alex Warren,https://open.spotify.com/track/3v9IFz7EikDfXXf...,Norway,9369.0,0.631,0.381,8.0,-8.232,0.0386,0.7680,0.002180,0.2610,0.5460,92.951,216935.0,Pop
57368,104549,Tomara,200,2021-05-14,"Mumuzinho, Matheus & Kauan",https://open.spotify.com/track/5ZcxuXjK5PmMoMc...,Brazil,93709.0,0.688,0.685,7.0,-7.699,0.0379,0.5040,0.000000,0.0984,0.8090,143.081,176298.0,Pop
57369,104551,Seribu Racun,200,2019-03-08,Barasuara,https://open.spotify.com/track/39g77d5bDuayJ0H...,Indonesia,17762.0,0.553,0.853,0.0,-4.307,0.0367,0.0010,0.011000,0.1150,0.4850,126.056,211814.0,Rock
57370,104552,Denk Maar Niet,200,2019-04-26,Sarita Lorena,https://open.spotify.com/track/5G1okMhYARR41Eq...,Netherlands,27670.0,0.801,0.488,11.0,-12.174,0.1020,0.0282,0.000277,0.1120,0.7330,106.071,169811.0,Rap


In [30]:
#Saving the sub dataset for testing.
from pathlib import Path
path = Path('../data/cleanedDataSetWithGenres.csv')
path.parent.mkdir(parents=True, exist_ok=True)
df_with_genre.to_csv(path)